<a href="https://colab.research.google.com/github/chcorophyll/my_deeplearning_cookbook/blob/master/my_Build_a_recommender_system_based_on_outgoing_Wikipedia_links.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!git clone https://github.com/chcorophyll/deep_learning_cookbook.git

Cloning into 'deep_learning_cookbook'...
remote: Enumerating objects: 427, done.
remote: Total 427 (delta 0), reused 0 (delta 0), pack-reused 427
Receiving objects: 100% (427/427), 160.26 MiB | 35.34 MiB/s, done.
Resolving deltas: 100% (207/207), done.
Checking out files: 100% (86/86), done.


In [34]:
!ls

'03.1 Using pre trained word embeddings.ipynb'
'03.2 Domain specific ranking using word2vec cosine distance.ipynb'
'04.1 Collect movie data from Wikipedia.ipynb'
'04.2 Build a recommender system based on outgoing Wikipedia links.ipynb'
'05.1 Generating Text in the Style of an Example Text.ipynb'
'06.1 Question matching.ipynb'
'07.1 Text Classification.ipynb'
'07.2 Emoji Suggestions.ipynb'
'07.3 Tweet Embeddings.ipynb'
'08.1 Sequence to sequence mapping.ipynb'
'08.2 Import Gutenberg.ipynb'
'08.3 Subword tokenizing.ipynb'
'09.1 Reusing a pretrained image recognition network.ipynb'
'09.2 Images as embeddings.ipynb'
'09.3 Retraining.ipynb'
'10.1 Building an inverse image search service.ipynb'
'11.1 Detecting Multiple Images.ipynb'
'12.1 Activation Optimization.ipynb'
'12.2 Neural Style.ipynb'
'13.1 Quick Draw Cat Autoencoder.ipynb'
'13.2 Variational Autoencoder.ipynb'
'13.5 Quick Draw Autoencoder.ipynb'
'14.1 Importing icons.ipynb'
'14.2 Icon Autoencoding.ipynb'
'14.2 Variational Autoencod

In [0]:
import os

path_o = os.getcwd()
data_path = os.path.join(path_o, "deep_learning_cookbook")
os.chdir(data_path)

In [0]:
!ls

'03.1 Using pre trained word embeddings.ipynb'
'03.2 Domain specific ranking using word2vec cosine distance.ipynb'
'04.1 Collect movie data from Wikipedia.ipynb'
'04.2 Build a recommender system based on outgoing Wikipedia links.ipynb'
'05.1 Generating Text in the Style of an Example Text.ipynb'
'06.1 Question matching.ipynb'
'07.1 Text Classification.ipynb'
'07.2 Emoji Suggestions.ipynb'
'07.3 Tweet Embeddings.ipynb'
'08.1 Sequence to sequence mapping.ipynb'
'08.2 Import Gutenberg.ipynb'
'08.3 Subword tokenizing.ipynb'
'09.1 Reusing a pretrained image recognition network.ipynb'
'09.2 Images as embeddings.ipynb'
'09.3 Retraining.ipynb'
'10.1 Building an inverse image search service.ipynb'
'11.1 Detecting Multiple Images.ipynb'
'12.1 Activation Optimization.ipynb'
'12.2 Neural Style.ipynb'
'13.1 Quick Draw Cat Autoencoder.ipynb'
'13.2 Variational Autoencoder.ipynb'
'13.5 Quick Draw Autoencoder.ipynb'
'14.1 Importing icons.ipynb'
'14.2 Icon Autoencoding.ipynb'
'14.2 Variational Autoencod

**Collect movie data from Wikipedia**

In [0]:
!pip install mwparserfromhell

     |████████████████████████████████| 143kB 4.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/2a/76/d5/7088b941df3b362c45dd7912dd05314bc034751ec9cbca9a75
Successfully built mwparserfromhell


In [0]:
import requests 
from bs4 import BeautifulSoup
import os
import time
from keras.utils import get_file
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve
import xml.sax
import subprocess
import re
import mwparserfromhell
import json

Using TensorFlow backend.


In [0]:
index = requests.get("https://dumps.wikimedia.org/enwiki/").text
soup_index = BeautifulSoup(index, "html.parser")
dumps = [a['href'] for a in soup_index.find_all("a") 
         if a.has_attr("href") and a.text[:-1].isdigit()]
dumps

['20190320/',
 '20190401/',
 '20190420/',
 '20190501/',
 '20190520/',
 '20190601/',
 '20190620/']

In [0]:
pages_xml = []
for dump_url in sorted(dumps, reverse=True):
    print(dump_url)
    dump_html = index = requests.get('https://dumps.wikimedia.org/enwiki/' + dump_url).text
    soup_dump = BeautifulSoup(dump_html, 'html.parser')
#     print(soup_dump)
    pages_xml += [a['href'] for a in soup_dump.find_all('a') 
                 if a.has_attr('href') and a['href'].endswith('-pages-articles.xml.bz2')]
#     if pages_xml:
#         break
    time.sleep(0.8)

20190620/
20190601/
20190520/
20190501/
20190420/
20190401/
20190320/


In [0]:
pages_xml

['/enwiki/20190601/enwiki-20190601-pages-articles.xml.bz2',
 '/enwiki/20190520/enwiki-20190520-pages-articles.xml.bz2',
 '/enwiki/20190501/enwiki-20190501-pages-articles.xml.bz2',
 '/enwiki/20190420/enwiki-20190420-pages-articles.xml.bz2',
 '/enwiki/20190401/enwiki-20190401-pages-articles.xml.bz2',
 '/enwiki/20190320/enwiki-20190320-pages-articles.xml.bz2']

In [0]:
wikipedia_dump = pages_xml[0].rsplit("/")[-1]
url = url = 'https://dumps.wikimedia.org/' + pages_xml[0] 
path = get_file(wikipedia_dump, url)
path

  658735104/16181808319 [>.............................] - ETA: 2:05:20

KeyboardInterrupt: ignored

In [0]:
def process_article(title, text):
    rotten = [(re.findall("\d\d?\d?%", p), re.findall("\d\.\d\/\d+|$", p),
               p.lower().find("rotten tomatoes")) for p in text.split("\n\n")]
    rating = next(((perc[0], rating[0]) for perc, rating, idx 
                   in rotten if len(perc) == 1 and idx > -1), (None, None))
    wikicode = mwparserfromhell.parse(text)
    film = next((template for template in wikicode.filter_templates() 
                 if template.name.strip().lower() == "infobox film"), None)
    if film:
        properties = {param.name.strip_code().strip(): param.value.strip_code().strip() 
                      for param in film.params
                      if param.value.strip_code().strip()}
        links = [x.title.strip_code().strip() for x in wikicode.filter_wikilinks()]
    return (title, properties, links) + rating

In [0]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._movies = []
        self._curent_tag = None

    def characters(self, content):
        if self._curent_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        if name in ('title', 'text'):
            self._curent_tag = name
            self._buffer = []

    def endElement(self, name):
        if name == self._curent_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            movie = process_article(**self._values)
            if movie:
                self._movies.append(movie)

In [0]:
parser = xml.sax.make_parser()
handler = WikiXmlHandler()
parser.setContentHandler(handler)
for line in subprocess.Popen(['bzcat'], stdin=open(path), stdout=subprocess.PIPE).stdout:
    try:
        parser.feed(line)
    except StopIteration:
        break

In [0]:
with open('generated/wp_movies.ndjson', 'wt') as fout:
    for movie in handler._movies:
         fout.write(json.dumps(movie) + '\n')

** Build a recommender system based on outgoing Wikipedia links**

In [0]:
import json
from collections import Counter
from keras.models import Model
from keras.layers import Embedding, Input, Reshape
from keras.layers.merge import Dot
from sklearn.linear_model import LinearRegression
import numpy as np
import random
from sklearn import svm

In [0]:
with open("data/wp_movies_10k.ndjson") as fin:
    movies = [json.loads(l) for l in fin]

In [38]:
movies[0]

['Deadpool (film)',
 {'Software Used': 'Adobe Premier Pro',
  'alt': "Official poster shows the titular hero Deadpool standing in front of the viewers, with hugging his hands, and donning his traditional black and red suit and mask, and the film's name, credits and billing below him.",
  'budget': '$58 million',
  'caption': 'Theatrical release poster',
  'cinematography': 'Ken Seng',
  'country': 'United States',
  'director': 'Tim Miller',
  'distributor': '20th Century Fox',
  'editing': 'Julian Clarke',
  'gross': '$783.1 million',
  'image': 'Deadpool poster.jpg',
  'language': 'English',
  'music': 'Tom Holkenborg',
  'name': 'Deadpool',
  'runtime': '108 minutes'},
 ['Tim Miller (director)',
  'Simon Kinberg',
  'Ryan Reynolds',
  'Lauren Shuler Donner',
  'Rhett Reese',
  'Paul Wernick',
  'Deadpool',
  'Fabian Nicieza',
  'Rob Liefeld',
  'Morena Baccarin',
  'Ed Skrein',
  'T.J. Miller',
  'Gina Carano',
  'Leslie Uggams',
  'Brianna Hildebrand',
  'Stefan Kapičić',
  'Junkie

In [39]:
len(movies)

10000

In [40]:
link_counts = Counter()
for movie in movies:
    link_counts.update(movie[2])
link_counts.most_common(10)

[('Rotten Tomatoes', 9393),
 ('Category:English-language films', 5882),
 ('Category:American films', 5867),
 ('Variety (magazine)', 5450),
 ('Metacritic', 5112),
 ('Box Office Mojo', 4186),
 ('The New York Times', 3818),
 ('The Hollywood Reporter', 3553),
 ('Roger Ebert', 2707),
 ('Los Angeles Times', 2454)]

In [41]:
top_links = [link for link, c in link_counts.items() if c >= 3]
link_to_idx = {link: idx for idx, link in enumerate(top_links)}
movie_to_idx = {movie[0]: idx for idx, movie in enumerate(movies)}
pairs = []
for movie in movies:
    pairs.extend((link_to_idx[link], movie_to_idx[movie[0]]) 
                 for link in movie[2] if link in link_to_idx)
pairs_set = set(pairs)
len(pairs), len(top_links), len(movie_to_idx)

(949544, 66913, 10000)

In [42]:
def movie_embedding_model(embedding_size=50):
    link = Input(name="link", shape=(1,))
    movie = Input(name="movie", shape=(1,))
    link_embedding = Embedding(name="link_enbedding", input_dim=len(top_links),
                               output_dim=embedding_size)(link)
    movie_embedding = Embedding(name="movie_enbedding", input_dim=len(movie_to_idx),
                               output_dim=embedding_size)(movie)
    dot = Dot(name="dot_product", normalize=True, 
              axes=2)([link_embedding, movie_embedding])
    merged = Reshape((1, ))(dot)
    model = Model(inputs=[link, movie], outputs=[merged])
    model.compile(optimizer="nadam", loss="mse")
    return model

model = movie_embedding_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
link (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
movie (InputLayer)              (None, 1)            0                                            
__________________________________________________________________________________________________
link_enbedding (Embedding)      (None, 1, 50)        3345650     link[0][0]                       
__________________________________________________________________________________________________
movie_enbedding (Embedding)     (None, 1, 50)        500000      movie[0][0]                      
__________________________________________________________________________________________________
dot_produc

In [43]:
random.seed(5)

def batchifier(pairs, positive_samples=50, negative_ratio=10):
    batch_size = positive_samples * (1 + negative_ratio)
    batch = np.zeros((batch_size, 3))
    while True:
        for idx, (link_id, movie_id) in enumerate(random.sample(pairs, positive_samples)):
            batch[idx, :] = (link_id, movie_id, 1)
        while idx < batch_size:
            movie_id = random.randrange(len(movie_to_idx))
            link_id = random.randrange(len(top_links))
            if not (link_id, movie_id) in pairs_set:
                batch[idx, :] = (link_id, movie_id, -1)
                idx += 1
        np.random.shuffle(batch)
        yield {"link": batch[:, 0], "movie": batch[:, 1]}, batch[:, 2]
next(batchifier(pairs, positive_samples=3, negative_ratio=2))            

({'link': array([31254., 32680., 32643., 32318., 13365., 48731., 20558., 22418.,
          3801.]),
  'movie': array([5530., 9403., 7628., 7685., 6238., 1854.,  849., 1529., 5874.])},
 array([ 1., -1., -1., -1., -1., -1., -1.,  1., -1.]))

In [44]:
positive_samples_per_batch = 512

model.fit_generator(batchifier(pairs, 
                               positive_samples=positive_samples_per_batch, 
                               negative_ratio=10),
                    epochs=15, 
                    steps_per_epoch=len(pairs) // positive_samples_per_batch,
                    verbose=2)

Epoch 1/15
 - 54s - loss: 0.3886
Epoch 2/15
 - 53s - loss: 0.2417
Epoch 3/15
 - 53s - loss: 0.2357
Epoch 4/15
 - 53s - loss: 0.2336
Epoch 5/15
 - 53s - loss: 0.2323
Epoch 6/15
 - 53s - loss: 0.2310
Epoch 7/15
 - 54s - loss: 0.2319
Epoch 8/15
 - 54s - loss: 0.2304
Epoch 9/15
 - 54s - loss: 0.2297
Epoch 10/15
 - 53s - loss: 0.2277
Epoch 11/15
 - 53s - loss: 0.2286
Epoch 12/15
 - 53s - loss: 0.2283
Epoch 13/15
 - 54s - loss: 0.2295
Epoch 14/15
 - 53s - loss: 0.2278
Epoch 15/15
 - 54s - loss: 0.2288


In [45]:
movie = model.get_layer("movie_enbedding")
movie_weights = movie.get_weights()[0]
movie_lengths = np.linalg.norm(movie_weights, axis=1)
normalized_movies = (movie_weights.T / movie_lengths).T

def similar_movies(movie):
    dists = np.dot(normalized_movies, normalized_movies[movie_to_idx[movie]])
    closet = np.argsort(dists)[-10:]
    for c in reversed(closet):
        print(c, movies[c][0], dists[c])
        
similar_movies('Rogue One')        

29 Rogue One 1.0000001
101 Prometheus (2012 film) 0.97176385
3349 Star Wars: The Force Awakens 0.9674347
19 Interstellar (film) 0.9650758
181 Pacific Rim (film) 0.95642644
22 Jurassic World 0.9562415
160 Jupiter Ascending 0.95237947
245 Gravity (film) 0.9506657
169 Alice in Wonderland (2010 film) 0.9504535
34 Doctor Strange (film) 0.9497459


In [46]:
movie_weights.shape

(10000, 50)

In [47]:
movie_weights[0].shape

(50,)

In [49]:
link = model.get_layer('link_enbedding')
link_weights = link.get_weights()[0]
link_lengths = np.linalg.norm(link_weights, axis=1)
normalized_links = (link_weights.T / link_lengths).T

def similar_links(link):
    dists = np.dot(normalized_links, normalized_links[link_to_idx[link]])
    closest = np.argsort(dists)[-10:]
    for c in reversed(closest):
        print(c, top_links[c], dists[c])

similar_links('George Lucas')

127 George Lucas 1.0000001
3176 Star Wars (film) 0.9450498
976 Hugo Award for Best Dramatic Presentation 0.93502736
4830 widescreen 0.9331149
2707 Star Wars 0.9302409
2778 Lucasfilm 0.92021817
3203 home video 0.92018425
1671 dailies 0.91508174
2721 Jaws (film) 0.9053166
3040 Simon & Schuster 0.90185064


In [50]:
best = ['Star Wars: The Force Awakens', 'The Martian (film)', 'Tangerine (film)', 'Straight Outta Compton (film)',
        'Brooklyn (film)', 'Carol (film)', 'Spotlight (film)']
worst = ['American Ultra', 'The Cobbler (2014 film)', 'Entourage (film)', 'Fantastic Four (2015 film)',
         'Get Hard', 'Hot Pursuit (2015 film)', 'Mortdecai (film)', 'Serena (2014 film)', 'Vacation (2015 film)']
y = np.asarray([1 for _ in best] + [0 for _ in worst])
X = np.asarray([normalized_movies[movie_to_idx[movie]] for movie in best + worst])
X.shape

(16, 50)

In [52]:
clf = svm.SVC(kernel="linear")
clf.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [54]:
estimated_movie_ratings = clf.decision_function(normalized_movies)
best = np.argsort(estimated_movie_ratings)
print('best:')
for c in reversed(best[-5:]):
    print(c, movies[c][0], estimated_movie_ratings[c])

print('worst:')
for c in best[:5]:
    print(c, movies[c][0], estimated_movie_ratings[c])

best:
70 Carol (film) 0.9999062031306364
307 Les Misérables (2012 film) 0.9963343321734829
481 The Devil Wears Prada (film) 0.9421521218876757
458 Hugo (film) 0.9345344633094338
66 Skyfall 0.9310475594488188
worst:
9890 Beach Party -1.5444213490393652
4100 See No Evil (2006 film) -1.5398974410194204
9595 Speed Zone -1.5246068130600527
4487 Tremors (franchise) -1.5145273638105368
7593 Trojan War (film) -1.4950135907038253


In [0]:
rotten_y = np.asarray([float(movie[-2][:-1])/100
                       for movie in  movies if movie[-2]])
rotten_X = np.asarray([normalized_movies[movie_to_idx[movie[0]]]
                       for movie in movies if movie[-2]])


In [56]:
TRAINING_CUT_OFF = int(len(rotten_X) * 0.8)
regr = LinearRegression()
regr.fit(rotten_X[:TRAINING_CUT_OFF], rotten_y[:TRAINING_CUT_OFF])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [57]:
error = (regr.predict(rotten_X[TRAINING_CUT_OFF:]) - rotten_y[TRAINING_CUT_OFF:])
'mean square error %2.2f' % np.mean(error ** 2)

'mean square error 0.06'

In [58]:
error = (np.mean(rotten_y[:TRAINING_CUT_OFF]) - rotten_y[TRAINING_CUT_OFF:])
'mean square error %2.2f' % np.mean(error ** 2)

'mean square error 0.09'

In [59]:
def gross(movie):
    v = movie[1].get('gross')
    if not v or not ' ' in v:
        return None
    v, unit = v.split(' ', 1)
    unit = unit.lower()
    if not unit in ('million', 'billion'):
        return None
    if not v.startswith('$'):
        return None
    try:
        v = float(v[1:])
    except ValueError:
        return None
    if unit == 'billion':
        v *= 1000
    return v

movie_gross = [gross(m) for m in movies]
movie_gross = np.asarray([gr for gr in movie_gross if gr is not None])
highest = np.argsort(movie_gross)[-10:]
for c in reversed(highest):
    print(c, movies[c][0], movie_gross[c])

6 The Martian (film) 10900.0
7 List of Marvel Cinematic Universe films 4300.0
49 Back to the Future 3900.0
71 The Conjuring 2932.0
162 Thor (film) 2464.0
36 Furious 7 2340.0
30 Finding Dory 2187.0
1906 Jane Eyre (2011 film) 2068.0
19 Interstellar (film) 1670.0
2251 An American Werewolf in London 1655.0


In [60]:
gross_y = np.asarray([gr for gr in movie_gross if gr])
gross_X = np.asarray([normalized_movies[movie_to_idx[movie[0]]] for movie, gr in zip(movies, movie_gross) if gr])

TRAINING_CUT_OFF = int(len(gross_X) * 0.8)
regr = LinearRegression()
regr.fit(gross_X[:TRAINING_CUT_OFF], gross_y[:TRAINING_CUT_OFF])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [61]:
error = (regr.predict(gross_X[TRAINING_CUT_OFF:]) - gross_y[TRAINING_CUT_OFF:])
'mean square error %2.2f' % np.mean(error ** 2)

'mean square error 8336.13'

In [62]:
error = (np.mean(gross_y[:TRAINING_CUT_OFF]) - gross_y[TRAINING_CUT_OFF:])
'mean square error %2.2f' % np.mean(error ** 2)

'mean square error 14115.59'